In [1]:
# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)

Pillow Version: 9.4.0


In [9]:
from PIL import Image

image = Image.open('data/train/dog.10025.jpg')

print(image.format)
print(image.mode)
print(image.size)

image.show()

JPEG
RGB
(499, 375)


In [4]:
import numpy as np

imdata = np.asarray(image)

print(imdata.shape)

image2 = Image.fromarray(imdata)

print(image2.format)
print(image2.mode)
print(image2.size)

(269, 259, 3)
None
RGB
(259, 269)


In [7]:
from PIL import ImageOps

def padding(image, expected_size):
    delta_width = expected_size[0] - image.size[0]
    delta_height = expected_size[1] - image.size[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(image, padding)

def resize_with_padding(image, expected_size):
    image.thumbnail((expected_size[0], expected_size[1]))

    delta_width = expected_size[0] - image.size[0]
    delta_height = expected_size[1] - image.size[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(image, padding)

In [14]:
from os import listdir
import matplotlib.image
# import matplotlib.image as img

loaded_images = list()
loaded_labels = list()
count = 0
for filename in listdir('data/train/'):
    if filename.find('cat') != -1:
        loaded_labels.append('cat')
    else:
        loaded_labels.append('dog')

    img = Image.open('data/train/' + filename)
    resized_img = resize_with_padding(img, (100, 100))

    img_data = np.asarray(resized_img)
    loaded_images.append(img_data)
    count += 1
    if count % 200 == 0:
        print("> loaded %d images" % (count))
    # print('> loaded %s %s' % (filename, img_data.shape))
print("Finished")

> loaded 200 images
> loaded 400 images
> loaded 600 images
> loaded 800 images
> loaded 1000 images
> loaded 1200 images
> loaded 1400 images
> loaded 1600 images
> loaded 1800 images
> loaded 2000 images
Finished


In [15]:
print(loaded_labels[100])
test_img = Image.fromarray(loaded_images[100])
test_img.show()

cat


## Data Manipulation

In [19]:
data_set = [list(a) for a in zip(loaded_images, loaded_labels)]
print(type(data_set))

<class 'list'>


In [22]:
# print(data_set[0])
print(data_set[0][0].shape)
print(data_set[0][1])

(100, 100, 3)
cat


## Define a CNN

In [16]:
import torch
import torchvision
import torch.nn as nn
import torch.functional as F

/home/hanhf/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/hanhf/anaconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            # [100, 100, 3]
            nn.Conv2d(
                in_channels=3,
                out_channels=6,
                kernel_size=5
            ),
            # [96, 96, 6]
            nn.MaxPool2d(
                kernel_size=4,
                stride=4
            ),
            # [24, 24, 6]
            nn.Conv2d(
                in_channels=6,
                out_channels=12,
                kernel_size=5
            ),
            # [20, 20, 12]
            nn.MaxPool2d(
                kernel_size=4,
                stride=4
            ),
            # [5, 5, 12]
            nn.Flatten(),
            # [5*5*12, 1]
            nn.Linear(
                in_features=5*5*12, 
                out_features=120
            ),
            # [1, 120]
            nn.ReLU(),
            nn.Linear(
                in_features=120, 
                out_features=30
            ),
            # [1, 30]
            nn.ReLU(),
            nn.Linear(
                in_features=30, 
                out_features=2
            )
            # [1, 2]
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x